# Decision Tree

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from numpy  import *

url='https://coding.net/u/HongHuangNeu/p/Machine-Learning-Notes-Data/git/raw/master/Lenses/lenses.txt'
df = pd.read_csv(url,sep='\t')
df[:3] #show the first 3 rows

,age,prescript,astigmatic,tearRate,label
0,young,myope,no,reduced,no lenses
1,young,myope,no,normal,soft
2,young,myope,yes,reduced,no lenses


Decision tree is very easy to understand. When applying a decision tree, users start from the root node. Each node of a decision tree is a question to ask about the feature values of the data point. Based on the different answers, users follow different branches to the next node. Eventually a leaf node will be reached, which contains the label of the test data point.

## ID3 Algorithm

This is a simple algorithm to construct decision tree. It only handles categorical features. In the decision tree constructed by this algorithm, each node is only about one feature of the data set, and each branch coming out of this node correspond to one value of this feature. A feature which has been used to split the data set will not be used again. 

This algorithm works by recursively do the following:

1. Calculate the entropy of the data set.
2. For each feature, calculate the sum of entropy $E_i$ after spliting the data set according to different values of feature $i$, then calculate the decrease of entropy with this splitting.
3. Choose the feature which leads to the maximum increase in entropy.
4. Apply the previous four steps recursively to the splitted data set.

Stop criteria:

1. When the splitted data set only contains objects from the same class, stop splitting. The class label of the corresponding node is the label of the objects.
2. When there is no feature left to split, stop splitting. The class label of the corresponding node is the majority label of the objects is the data set.

## Entropy
Entropy describes how messy a system is. The higher the value, the more messy it is. Here the notion of entropy is used to describe the distribution of class labels. A data set is "messy" if it contains objects from different classes. 

Suppose a data set contains objects from $M$ different classes $C_1,C_2,...C_m$. The entropy of the data set is:

$$H(C)=-\sum^{m}_{i=1}P(C_i)\log_2 P(C_i)$$

This formula is used to calculate the entropy before splitting the data set. If the data set only contains objects with one label, the entropy is 0.

## Information Gain
By splitting the data set, we want to decrease the entropy as much as possible. We quantify the decrease with the notion of **Information Gain**, which is defined as follows:
$$IG(S)=H(C)-\sum^{k}_{i=1}\frac{|D_i|}{|D|} H(D_i)$$

Here we suppose feature $S$ has $k$ distinct values $v_1,v_2,...v_k$. $H(C)$ is the entropy before splitting the data set. $IS(S)$ is the information gain by splitting on feature $S$. Each subset $D_i$ of the original set only contain onjects with value $v_i$ on feature $S$. The information gain is the difference between the entropy before splitting and the sum of weighted entropy after splitting. 

The ID3 algorithm always choose to split on the feature which gives the biggest **Information Gain**.


## Implementation
Given all these ingredients, we could srat implementing the ID3 algorithm.
### Entropy of the Original Set
To calculate the entropy of a data set, just enumerate each object in the data set and do counting.

In [3]:
from math import log
def entropy(dataSet):
    labelCount = {}
    numberOfRows = dataSet.shape[0]
    for data in dataSet:
        label = data[-1] #we assume that the last column is the label column.
        if label in labelCount.keys():
            labelCount[label] = labelCount[label]+1
        else:
            labelCount[label] = 1
    entropy = 0.0
    for key in labelCount.keys():
        proportion=float(labelCount[label])/numberOfRows
        entropy = entropy - proportion * log(proportion,2)
    return entropy

#play with this function
entropy(df.values) #notice that df is a pandas data frame, needs to be converted to numpy array first


1.2713848220861959

### Data Set Split on a Feature Value
Given a value of a feature, we need to have a subset of the original data set with objects with only this value on this feature. This function will be used to split a data set on a given feature by the distinct values of this feature. A new numpy array will be created for this purpose.

In [29]:
import numpy as np
def splitData(dataSet,featureIndex,value):
    containData=False
    resultDataSet=array([])
    
    #if the resultDataSet does not have any value, assign the feature vector to it, otherwise just append
    for data in dataSet:
        if(data[featureIndex] == value):
            resultFeature = data[:featureIndex]
            resultFeature = np.append(resultFeature,data[featureIndex+1:]) 
            if containData == False:
                #we need to remove this column before adding it to the result                
                resultDataSet = resultFeature
                containData = True
            else:
            #add this feature to result
                resultDataSet = np.vstack((resultDataSet, resultFeature))
            #resultDataSet.append(resultFeature)
    if resultDataSet.shape[0] == 1:
        resultDataSet = resultDataSet.reshape(1,dataSet.shape[1]-1)
    return resultDataSet
a=splitData(df.values,0,'young')

a

array([['myope', 'no', 'reduced', 'no lenses'],
       ['myope', 'no', 'normal', 'soft'],
       ['myope', 'yes', 'reduced', 'no lenses'],
       ['myope', 'yes', 'normal', 'hard'],
       ['hyper', 'no', 'reduced', 'no lenses'],
       ['hyper', 'no', 'normal', 'soft'],
       ['hyper', 'yes', 'reduced', 'no lenses'],
       ['hyper', 'yes', 'normal', 'hard']], dtype=object)

### Choose the Best Split of Data
First, make the function that splits on a particular feature

In [38]:
matrix=df.values
matrix[:,3]
se=[str(s) for s in matrix[:,4]]
set(se)
#calculate the new entropy after splitting on a feature
def EntropyOnFeature(dataSet,featureIndex):
    list_of_feature_values = [str(s) for s in dataSet[:,featureIndex]]
    distinct_values = set(list_of_feature_values)
    numberOfRows = dataSet.shape[0]
    entropySum = 0
    for value in distinct_values:
        newSet = splitData(dataSet,featureIndex,value)
        e = entropy(newSet)
        entropySum = entropySum + float(newSet.shape[0])/float(numberOfRows)*e
    return entropySum

EntropyOnFeature(df.values,1)

1.3349625007211563

Then, enumerate on the features and choose the best one

In [43]:
def chooseBestFeature(dataSet):
    numberOfFeatures = dataSet.shape[1]-1
    bestFeatureIndex = -1
    bestInfoGain = 0
    baseEntropy = entropy(dataSet)
    for i in range(numberOfFeatures):
        newEntropy = EntropyOnFeature(dataSet,i)
        infoGain = baseEntropy - newEntropy
        print(str(dataSet))
        print("base entropy"+str(baseEntropy))
        print("new entropy"+str(newEntropy))
        print("index"+str(i))
        if infoGain > bestInfoGain:
            bestInfoGain = infoGain
            bestFeatureIndex = i
    return bestFeatureIndex

chooseBestFeature(df.values) #the best feature should be tearRate

[['young' 'myope' 'no' 'reduced' 'no lenses']
 ['young' 'myope' 'no' 'normal' 'soft']
 ['young' 'myope' 'yes' 'reduced' 'no lenses']
 ['young' 'myope' 'yes' 'normal' 'hard']
 ['young' 'hyper' 'no' 'reduced' 'no lenses']
 ['young' 'hyper' 'no' 'normal' 'soft']
 ['young' 'hyper' 'yes' 'reduced' 'no lenses']
 ['young' 'hyper' 'yes' 'normal' 'hard']
 ['pre' 'myope' 'no' 'reduced' 'no lenses']
 ['pre' 'myope' 'no' 'normal' 'soft']
 ['pre' 'myope' 'yes' 'reduced' 'no lenses']
 ['pre' 'myope' 'yes' 'normal' 'hard']
 ['pre' 'hyper' 'no' 'reduced' 'no lenses']
 ['pre' 'hyper' 'no' 'normal' 'soft']
 ['pre' 'hyper' 'yes' 'reduced' 'no lenses']
 ['pre' 'hyper' 'yes' 'normal' 'no lenses']
 ['presbyopic' 'myope' 'no' 'reduced' 'no lenses']
 ['presbyopic' 'myope' 'no' 'normal' 'no lenses']
 ['presbyopic' 'myope' 'yes' 'reduced' 'no lenses']
 ['presbyopic' 'myope' 'yes' 'normal' 'hard']
 ['presbyopic' 'hyper' 'no' 'reduced' 'no lenses']
 ['presbyopic' 'hyper' 'no' 'normal' 'soft']
 ['presbyopic' 'hype

3

### Create the Tree Recursively
The tree will be stored in a nested dict. The top most key would be the name of the first feature to check. The value would be the values of this feature. Each feature value correspond to a new dict (or a class label) which again be the name of the feature to check next. So the key would be feature name->feature value->feature name......

The tree will be created recursively: First create the node of the parent, then recursively create sub-trees of children.

In [66]:
def createDecisionTree(dataSet, columnNames):
    resultDict={}
    print(str(dataSet.shape))
    print(str(dataSet))
    numberOfColumns = dataSet.shape[1]
    list_of_label_values = [str(s) for s in dataSet[:,numberOfColumns-1]]
    
    distinct_values = set(list_of_label_values)
    print(" set of labels "+str(distinct_values))
    if len(distinct_values) == 1:
        return list_of_label_values[0]
    
    if numberOfColumns == 1:
        countDict = {}
        for label in list_of_label_values:
            if label in countDict:
                countDict[label] = countDict[label] + 1
            else:
                countDict[label] = 1
        sortedClassLabels = sorted(countDict.items(),key=lambda x: x[1],reverse=True)
        return sortedClassLabels[0][0]
    
    bestFeatureIndex = chooseBestFeature(dataSet)
#    if bestFeatureIndex == -1:
#        vcountDict = {}
#        for label in list_of_label_values:
#            if label in vcountDict:
#                vcountDict[label] = vcountDict[label] + 1
#            else:
#                vcountDict[label] = 1
#        vsortedClassLabels = sorted(vcountDict.items(),key=lambda x: x[1],reverse=True)
#        return vsortedClassLabels[0][0]
    print("column names"+str(columnNames))
    print("number of columns"+str(numberOfColumns) )
    print("chosen index"+str(bestFeatureIndex))    
    bestFeatureName = columnNames[bestFeatureIndex]
    print("chosen "+bestFeatureName)
    list_of_feature_values = [str(s) for s in dataSet[:,bestFeatureIndex]]
    distinct_feature_values = set(list_of_feature_values)
    resultDict[bestFeatureName]={}
    cols=columnNames.copy()
    del(cols[bestFeatureIndex])
    for s in distinct_feature_values:
        splittedDataSet = splitData(dataSet,bestFeatureIndex,s)       
        resultDict[bestFeatureName][s] = createDecisionTree(splittedDataSet,cols)
    return resultDict
    
createDecisionTree(df.values,list(df.columns.values))

(24, 5)
[['young' 'myope' 'no' 'reduced' 'no lenses']
 ['young' 'myope' 'no' 'normal' 'soft']
 ['young' 'myope' 'yes' 'reduced' 'no lenses']
 ['young' 'myope' 'yes' 'normal' 'hard']
 ['young' 'hyper' 'no' 'reduced' 'no lenses']
 ['young' 'hyper' 'no' 'normal' 'soft']
 ['young' 'hyper' 'yes' 'reduced' 'no lenses']
 ['young' 'hyper' 'yes' 'normal' 'hard']
 ['pre' 'myope' 'no' 'reduced' 'no lenses']
 ['pre' 'myope' 'no' 'normal' 'soft']
 ['pre' 'myope' 'yes' 'reduced' 'no lenses']
 ['pre' 'myope' 'yes' 'normal' 'hard']
 ['pre' 'hyper' 'no' 'reduced' 'no lenses']
 ['pre' 'hyper' 'no' 'normal' 'soft']
 ['pre' 'hyper' 'yes' 'reduced' 'no lenses']
 ['pre' 'hyper' 'yes' 'normal' 'no lenses']
 ['presbyopic' 'myope' 'no' 'reduced' 'no lenses']
 ['presbyopic' 'myope' 'no' 'normal' 'no lenses']
 ['presbyopic' 'myope' 'yes' 'reduced' 'no lenses']
 ['presbyopic' 'myope' 'yes' 'normal' 'hard']
 ['presbyopic' 'hyper' 'no' 'reduced' 'no lenses']
 ['presbyopic' 'hyper' 'no' 'normal' 'soft']
 ['presbyopi

{'tearRate': {'normal': {'astigmatic': {'no': {'age': {'pre': 'soft',
      'presbyopic': {'prescript': {'hyper': 'soft', 'myope': 'no lenses'}},
      'young': 'soft'}},
    'yes': {'prescript': {'hyper': {'age': {'pre': 'no lenses',
        'presbyopic': 'no lenses',
        'young': 'hard'}},
      'myope': 'hard'}}}},
  'reduced': 'no lenses'}}

## Visualize Decision Tree

In the book **Machine Learning in Action**, visualize the decision tree is way more complex than understanding and building the tree itself. Let's not waste time on that. Here we print it out in an understanderable way. First let's build an example of tree to visualize.

In [67]:
tree={}
tree['no surfacing']={}
tree['no surfacing'][0]='no'
tree['no surfacing'][1]={}
tree['no surfacing'][1]['flippers']={}
tree['no surfacing'][1]['flippers'][0]='no'
tree['no surfacing'][1]['flippers'][1]='yes'
tree

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

Now we visualize the tree recursively. For every node and every feature value we start a new line. The feature name and values are preceeded with "+". The number of "+" is determined by the level of the node and feature value. The feature name and the corresponding feature value comes at the same level. 

Once we enter a new subtree, the first key is always the feature name to check (because of the way we recurse). We print out this feature name, with the right number of preceeding "+". Then we enumerate all the values of this feature. For each feature value, we print them with the same number of preceeding "+" and then go into the subtree. If the object corresponding to the feature value is not a dict, it is a label, and we just print it out. 

In [68]:
#This function print text with a specified number of "+"
def printSeq(text,numberOfWhiteSpaces):
        s=" "
        for i in range(numberOfWhiteSpaces):
            s=s+" +"
        print(s,text)    
def printTree(tree, numberOfWhiteSpaces):
    if type(tree).__name__=='dict':
        firstStr=list(tree.keys())[0] #name of feature to check
    printSeq(firstStr,numberOfWhiteSpaces)    
    secondDict=tree[firstStr]
    for key in secondDict.keys(): #This time, each key is a feature value
        printSeq("value: "+str(key),numberOfWhiteSpaces)
        if type(secondDict[key]).__name__=='dict':          
            printTree(secondDict[key],numberOfWhiteSpaces+1)
        else:
            printSeq("label: "+secondDict[key],numberOfWhiteSpaces+1)

In [69]:
printTree(tree,0)

  no surfacing
  value: 0
  + label: no
  value: 1
  + flippers
  + value: 0
  + + label: no
  + value: 1
  + + label: yes


In [70]:
printTree(result,0)

  tearRate
  value: reduced
  + label: no lenses
  value: normal
  + astigmatic
  + value: yes
  + + prescript
  + + value: hyper
  + + + age
  + + + value: young
  + + + + label: hard
  + + + value: pre
  + + + + label: no lenses
  + + + value: presbyopic
  + + + + label: no lenses
  + + value: myope
  + + + label: hard
  + value: no
  + + age
  + + value: young
  + + + label: soft
  + + value: pre
  + + + label: soft
  + + value: presbyopic
  + + + prescript
  + + + value: hyper
  + + + + label: soft
  + + + value: myope
  + + + + label: no lenses
